In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [66]:
from typing import List, Dict
import csv
import time
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

In [17]:
data = pd.read_csv('/content/gdrive/MyDrive/IT-H/data_1.csv')
data = data[data['Role Category'] == 'Programming & Design'].dropna(subset = ['Key Skills'])

In [62]:
data.head(5)

,Uniq Id,Crawl Timestamp,Job Title,Job Salary,Job Experience Required,Key Skills,Role Category,Location,Functional Area,Industry,Role
4,2d8b7d44e138a54d5dc841163138de50,2019-07-05 02:48:29 +0000,Software Test Engineer -hyderabad,Not Disclosed by Recruiter,2 - 5 yrs,manual testing| test engineering| test cases|...,Programming & Design,Hyderabad,IT Software - QA & Testing,"IT-Software, Software Services",Testing Engineer
5,c9628ea8d9cfd2ca72e14d53783aca90,2019-08-05 03:59:27 +0000,Opening For Adobe Analytics Specialist,"7,00,000 - 17,00,000 PA.",5 - 7 yrs,adobe experience manager| digital| digital ma...,Programming & Design,Pune,"IT Software - Application Programming , Mainte...","IT-Software, Software Services",System Analyst
7,9642bed00e26183e41ab73778bf48108,2019-08-04 03:31:17 +0000,Opportunity For Azure Devops Architect For Hy...,Not Disclosed by Recruiter,9 - 14 yrs,TFS| Azure| Git| VSTS| Docker| DynaTrace| Spl...,Programming & Design,Hyderabad,"IT Software - Application Programming , Mainte...","IT-Software, Software Services",Technical Architect
13,225c4680e3df32fbaf9c625ca18ae47c,2019-08-04 09:37:17 +0000,Oracle IDAM,Openings: 1,3 - 7 Years,Oracle IDAM|OIM|OAM,Programming & Design,"Bengaluru,Hyderabad,Kolkata,Chennai","IT Software - Application Programming , Mainte...",IT-Software / Software Services,Software Developer
18,201088d1d596841523051e20d56611f5,2019-08-04 20:46:52 +0000,Magento Developer,Not Disclosed by Recruiter,2 - 7 yrs,Copyright| Email| jQuery| XML| Javascript| Ca...,Programming & Design,Ahmedabad,"IT Software - eCommerce , Internet Technologies","Textiles, Garments, Accessories",Software Developer


In [70]:
unique_skills = set()
for info in data['Key Skills']:
  skills = set(i.strip().lower() for i in info.split("|") if len(i.strip()))
  unique_skills |= skills


In [71]:
unique_skills = list(unique_skills)
comparisor = {skill: ind for ind, skill in enumerate(unique_skills)}

In [100]:
matrix = [[0 for i in range(len(comparisor))] for j in range(len(comparisor))]
for info in tqdm(data['Key Skills']):
  skills = [i.strip().lower() for i in info.split("|") if len(i.strip())]
  for skill1_ind in range(len(skills)):
    skill1 = comparisor[skills[skill1_ind]]
    for skill2_ind in range(skill1_ind, len(skills)):
      skill2 = comparisor[skills[skill2_ind]]
      matrix[skill1][skill2]+=1
      if skill1!=skill2:
        matrix[skill2][skill1]+=1
normalized_matrix = []
for row in matrix:
  summ = sum(row)
  normalized_matrix.append((np.array(row)/summ).tolist())
with open('/content/gdrive/MyDrive/IT-H/tech_matrix.npy', 'wb') as f:
    np.save(f, np.array(normalized_matrix))
with open('/content/gdrive/MyDrive/IT-H/skills.txt', 'w') as f:
    for skill in unique_skills:
      f.write(skill + '\n')

  0%|          | 0/8365 [00:00<?, ?it/s]

In [101]:
class Recommender():
  def __init__(self, source_path ='/content/gdrive/MyDrive/IT-H/'):
    with open(source_path + 'tech_matrix.npy', 'rb') as f:
      self.matrix = np.load(f)
    with open(source_path + 'skills.txt', 'r') as f:
      self.skills_list = [i.strip() for i in f.readlines()]
    self.comparator = {skill: i for i, skill in enumerate(self.skills_list)}
    self.skills_number = len(self.comparator)

  def search_candidates(self, tag: str):
    try:
      tag = self.comparator[tag.lower()]
    except:
      print('This tag ({}) is not in database'.format(tag))
      return {}
    candidates = {}
    for skill_ind in range(self.skills_number):
        if self.matrix[skill_ind][tag]:
          candidates[skill_ind] = self.matrix[skill_ind][tag]
    return candidates

  def merge_dicts(self, vocab1: Dict, vocab2: Dict) -> Dict:
    for key in vocab2:
      if key not in vocab1:
        vocab1[key] = 0
      vocab1[key]+=vocab2[key]
    return vocab1

  def predict(self, tags_list: List[str], topk = 3) -> List[str]:
    tags_list = [i.lower() for i in tags_list]
    candidates = {}
    for tag in tags_list:
      candidates = self.merge_dicts(candidates, self.search_candidates(tag))      
    possibles_rated = sorted(list(candidates.keys()), key = lambda x: -candidates[x])
    rated_as_text = list(filter(lambda skill: skill not in tags_list, [self.skills_list[skill_ind] for skill_ind in possibles_rated]))
    return rated_as_text[:topk]


In [102]:
Recommender().predict(['SQL', 'Python', 'Docker'])

['shell.', 'network protocol testing', 'python developers']

In [103]:
Recommender().predict(['js', 'html', 'css'])

['javascript.', 'html programmers', 'pre']

In [106]:
Recommender().predict(['ux', 'UI'])

['health care services', 'object oriented javascript', 'sketch']

In [107]:
Recommender().predict(['c++'])

['hil', 'embedded system design', 'sr associate']